Tagged: nikola, blog, jupyter, colab

---



## Introduction
Having recently completed my PhD thesis work, I have begun to shift gears in terms of the type of my research and training. For the last 15 years in the biological sciences I have been primarily at the bench performing 'wet' lab experiments and only supporting them a small fraction of the time using computational approaches. Now as I begin my post doctoral training my focus will be in the persuit of applying computational and bioinformatics approaches full-time. To facilitate this I have decided to document the process and the recipes gathered along the way by means of this static blog site deployed from GitHub and managed using the Python package Nikola.  

### Why a static site?  
A static site is free from server side code and has minimal hosting requirements.  This allows for a lightweight site based on mostly pure HTML. Many hosting solutions will also serve your static website for free such as GitHub and DigitalOcean.

### Site requirements
There are multiple goals for this project that require proper documentation of coding, pipelines, and workflows related to bioinformatics. Therefore I came up with several required capabilities for the blog site.

*   static site (pre-generated HTML with no need for server side processing)
*   ability to post formated highlighted code and output
*   support for displaying multiple languages (Python, R, bash, etc.)
*   ability to use multiple source documents (reStructured txt, markdown, Jupyter .ipynb)  
* deploy from GitHub (CI/CD)
---

## The Setup
Ultimately the tool that fit all of the requirements above was [Nikola](https://getnikola.com/). This is a static site generator that is fully featured with many plugins. Basically it is a command line tool that manages local folders on your computer. You create a blog post using reStructured txt, markdown, or Jupyter .ipynb and then using the command line tool add it to the Nikola managed site source. Luckily all the features we need come built-in without having to install any additional plugins.

### Install Nikola
I use [pipx](https://github.com/pypa/pipx) as a way to isolate python packages into their own environment and run them as command line tools. It is essentially python pip but installs each package and its requirements in a way that doesn't conflict with anything else on the system. Pipx also places executables and scripts in your path so that they can be run on the command line. (In the future I will dive deeper into my local setup.) Installing Nikola with pipx is as simple as:

`pipx install Nikola[extras]`  
`pipx inject Nikola ghp-import --include-apps --include-deps`

* `[extras]` allows some bonus features like .ipynb support.  
* `pipx inject Nikola ghp-import` adds the extra package `ghp-import` required for `github_deploy` directly into the same virtual environment with Nikola.

### Initialization
Initializing a new blog is as simple as creating a new directory, navigating to it and running:  

`nikola init`

This command will walk through asking questions setting up things like blog name, authors, email, and language.  

The end result of this step will create several folders and files that make up and manage the enitre blog. Importantly the `conf.py` is the configuration file that will be used to control everything from the themes, adding pages, and customizing the git settings. Couple of other folder related things to note relevant to us:
* `posts` - Once imported or created this is the folder where it will store the raw posts after they are imported. Edit these files in the future to change or update the posts.
* `pages` - Similar rules to the `posts` folder. This is the location for non-blog related pages such as About, Information, Contact, etc.
* `output` - After the `nikola build` command has been run the `output` folder will be propagated with the static html files that will be the final product rendered as the blog site. This needs to rebuilt with the `build` command when any posts or pages are updated or new content imported.

### Create a git repository
Within the newly initialized Nikola folder a git repository will be setup to track development changes and for automatic deployment from GitHub.

`git init . `  
`git remote add origin https://github.com/mlkaufman/bioinformaticslife-blog.git`

Create a `.gitignore` file and add the following lines: 

```
  cache  
  .doit.db*  
  __pycache__  
  output  
```

### Configuration and Customization
To setup the majority of the site I distilled several tutorials starting with the one at [https://jiaweizhuang.github.io/blog/nikola-guide/](https://jiaweizhuang.github.io/blog/nikola-guide/). Customization of the blog generated by Nikola is done through editing the `config.py` file found in the root directory created upon initialization. The [Nikola handbook](https://getnikola.com/handbook.html) is a great reference. [http://www.jaakkoluttinen.fi/blog/how-to-blog-with-jupyter-ipython-notebook-and-nikola/](http://www.jaakkoluttinen.fi/blog/how-to-blog-with-jupyter-ipython-notebook-and-nikola/) was another handy tutorial for setting up the page to work with `github_deploy`.  
  
All of the following configurations require modification of fields in the `conf.py` file.

#### Themes
Themes are easily chosen or swapped out on the fly. A gallery of themes to preview can be viewed [here](https://themes.getnikola.com/).

In [ ]:
# Name of the theme to use.
THEME = "bootstrap3"

# A theme color. In default themes, it might be displayed by some browsers as
# the browser UI color (eg. Chrome on Android). Other themes might also use it
# as an accent color (the default ones don’t). Must be a HEX value.
THEME_COLOR = '#5670d4'

#### Adding .ipynb Support
Added lines to both the POSTS and PAGES sections to allow import handling of .ipynb files. Otherwise support is built in, why not have this in the configuration by default?

In [ ]:
POSTS = (
    ("posts/*.rst", "blog", "post.tmpl"),
    ("posts/*.md", "blog", "post.tmpl"),
    ("posts/*.txt", "blog", "post.tmpl"),
    ("posts/*.html", "blog", "post.tmpl"),
    ("posts/*.ipynb", "blog", "post.tmpl"),) # added ipynb support

PAGES = (
    ("pages/*.rst", "", "page.tmpl"),
    ("pages/*.md", "", "page.tmpl"),
    ("pages/*.txt", "", "page.tmpl"),
    ("pages/*.html", "", "page.tmpl"),
    ("pages/*.ipynb", "", "page.tmpl"),) # added ipynb support

#### Getting rid of the Jupyter prompt box next to each cell
A little hack to remove the Jupyter prompt from each of the cells. These are the [ ] boxes that are present while running the cells and is rendered as an annoying grey box in the HTML output. Removed them to make the blog look prettier. Got the styling code from [here](https://www.reddit.com/r/IPython/comments/8linhh/hide_ln_out_at_the_left_of_each_cell/).

In [ ]:
EXTRA_HEAD_DATA = "<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>"

#### GitHub Config
The source and deploy/output branches are listed here so that Nikola knows where to push them to for rendering the site. It pushes the source as well so a copy is kept on GitHub and version controlled.

In [ ]:
#### GitHub Support
GITHUB_SOURCE_BRANCH = 'src'
GITHUB_DEPLOY_BRANCH = 'main'

# The name of the remote where you wish to push to, using github_deploy.
GITHUB_REMOTE_NAME = 'origin'

# Whether or not github_deploy should commit to the source branch automatically
# before deploying.
GITHUB_COMMIT_SOURCE = True

#### Adding a Comment System
Nikola supports linking many different 3rd party comment systems. Went with [Disqus](https://disqus.com/) here.

In [ ]:
COMMENT_SYSTEM = "Disqus"
# And you also need to add your COMMENT_SYSTEM_ID which
# depends on what comment system you use. The default is
# "nikolademo" which is a test account for Disqus. More information
# is in the manual.
COMMENT_SYSTEM_ID = ""

#### Add CNAME file
This command will add a CNAME file to the files folder that will redirect the webaddress without the `www` to the main site.  

`echo "bioinformaticslife.com" > files/CNAME`

---
## Write - Build - Deploy Cycle
Now that everything is configured, here I describe the workflow of producing an actual blog post.
### Write
As mentioned I really wanted to get the Jupyter .ipynb working as an option to make posts. This allows the direct posting of bioinformatics workflows and outputs. I played around with the idea of using a local installation of Jupyter and either using VSCode or the native web interface to interact with the Jupyter kernel and generate notebooks. I may still do this periodically in the future but for now I have settled on using [Google Colab](https://colab.research.google.com/).  
* always in the cloud and on every computer in a browser
* nice markdown editor and visualization
* easy to install additional packages with `!pip install` magic commands 
* can process large files placed in Google Drive 

### Build
After saving the notebook locally on the computer it can be imported to the initialized blog, the post can be added with the `nikola new_post [options] [path]` command.  

Incorporating a new post with the Jupyter notebook page looks something like this format:  

`nikola new_post -t <title of the post> -f <format> --tags <tags that describe the post> -i <import file.ipynb>`

Example:  

`nikola new_post -t "Helloish World" -f ipynb --tags "new,hello world,first post" -i ../helloworld.ipynb`

This gets the post copied into the source files of the local blog. Next these changes need to be built and incorporated into the existing HTML documents and structure. This is as simple as running:  
`nikola build`

Finally you can then preview these changes and the new post on the test server.  This command will load the test server and open a web browser to preview the site:  
`nikola serve -b`

### Depoly
Deploying the new blog post changes to GitHub and updating the live site should now be as simple as running the command:  
`nikola github_deploy`

This will automatically commit the changes and push them to GitHub. It stores the source files in the `src` branch while pushing the front facing webpages to `main` as we set in the config file. A commit message can be added using the `-m` flag. In the future if the local copy of the site needs to be replaced or copied to a new location, simply clone the `src` branch from GitHub and begin the Write-Build-Deploy cycle again.


---

## Final Thoughts
So far I was able to customize the site the way I wanted. The Write-Build-Deploy work cycle seems to enable a smooth way to post to the site. I will continue using this setup to experiment with different formats and documenting my progress. Stay tuned!
